# Tabular Dataset Exercise Solution
### The dataset is comming from this [Kaggle competition](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)
### Where the goal is to predict the price of houses in King Country, USA.
#### You can check the documentation of [Autogluon](https://auto.gluon.ai/stable/index.html)

In [ ]:
# If running on your own computer please refer to AutoGluon installation instructions:
# https://auto.gluon.ai/stable/install.html
# This notebook assumes running in SageMaker Studio with "PyTorch 1.12 Python 3.8 CPU Optimized" kernel.
!pip3 install autogluon
!pip3 install ipywidgets

#### 1) Run Autogluon default settings on this dataset

In [2]:
### import autogluon tabular predictor and datasets
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

In [3]:
try:
    dataset = TabularDataset('Datasets/kc_house_data.csv')
    train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=402)
except:
    # unzip dataset
    !unzip Datasets/archive.zip -d Datasets
    dataset = TabularDataset('Datasets/kc_house_data.csv')
    train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=402)
print(train_data.shape)
train_data.head()

(15129, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
10549,9477201370,20150226T000000,440000.0,3,1.75,1760,8025,1.0,0,0,...,7,1230,530,1976,0,98034,47.7287,-122.192,1590,7543
15154,2770604615,20141121T000000,735000.0,4,2.00,1640,6000,1.5,0,0,...,9,1640,0,1911,0,98119,47.6505,-122.375,1900,6000
20283,662440020,20150226T000000,380000.0,4,2.50,2420,4981,2.0,0,0,...,9,2420,0,2009,0,98038,47.3785,-122.023,2420,5000
7865,3211100450,20140814T000000,217000.0,3,1.00,1400,7800,1.0,0,0,...,7,1400,0,1962,0,98059,47.4789,-122.159,1400,7800
1614,2624049117,20140903T000000,425000.0,3,1.00,1550,4160,1.5,0,0,...,6,1550,0,1926,0,98118,47.5387,-122.265,1590,5000


In [4]:
label = 'price'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    1.512900e+04
mean     5.410016e+05
std      3.701607e+05
min      7.500000e+04
25%      3.200000e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64


### Call tabular predictor with default settings

In [5]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20221128_020152/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221128_020152/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7700000.0, 75000.0, 541001.55523, 370160.66869)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4282.14 MB
	Train Data (Original)  Memory Usage: 3.39 MB (0.1% of available memory)
	

[1000]	valid_set's rmse: 135959
[2000]	valid_set's rmse: 134862


	-134772.2801	 = Validation score   (-root_mean_squared_error)
	4.38s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 131760


	-131021.0799	 = Validation score   (-root_mean_squared_error)
	2.98s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-133070.6265	 = Validation score   (-root_mean_squared_error)
	16.75s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: CatBoost ...
	-124688.2758	 = Validation score   (-root_mean_squared_error)
	24.91s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-140276.7061	 = Validation score   (-root_mean_squared_error)
	8.09s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-113839.5512	 = Validation score   (-root_mean_squared_error)
	24.07s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: XGBoost ...
	-122627.4831	 = Validation score   (-root_mean_squared_error)
	13.94s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-174614.2544	 = Validation score   (-root_mean_squared_error)
	26.45s	 = Trai

### Performance evaluation

In [6]:
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2453,2111010340,20140612T000000,4,2.5,2490,8124,2.0,0,0,3,7,2490,0,2001,0,98092,47.3347,-122.170,2760,6300
9453,7215730040,20140515T000000,4,3.0,3150,9130,2.0,0,0,3,9,3150,0,2001,0,98075,47.5974,-122.017,2970,6228
809,1786640070,20140806T000000,3,2.0,1950,8698,1.0,0,0,3,8,1950,0,1999,0,98042,47.3900,-122.153,2330,7212
3109,2641800015,20141124T000000,3,1.0,960,8291,1.0,0,0,3,6,960,0,1950,0,98146,47.5006,-122.334,1110,8231
15286,7011201306,20150212T000000,4,4.0,2530,1774,3.0,0,2,3,9,2100,430,2013,0,98119,47.6362,-122.369,2160,2400


In [7]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -110553.98521752375
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -110553.98521752375,
    "mean_squared_error": -12222183647.476954,
    "mean_absolute_error": -62696.65711992983,
    "r2": 0.9056622735920185,
    "pearsonr": 0.952238654503172,
    "median_absolute_error": -37306.859375
}


### Models Leaderboard 

In [8]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-110553.985218,-108453.564605,1.293147,0.203325,41.406445,0.012414,0.000459,0.417148,2,True,12
1,CatBoost,-115355.709919,-124688.275829,0.058943,0.013872,24.906297,0.058943,0.013872,24.906297,1,True,6
2,NeuralNetFastAI,-116296.056824,-113839.551183,0.290310,0.079813,24.073407,0.290310,0.079813,24.073407,1,True,8
3,LightGBMXT,-117533.976782,-134772.280084,0.953617,0.110065,4.379845,0.953617,0.110065,4.379845,1,True,3
4,LightGBM,-119447.773607,-131021.079891,0.753550,0.073037,2.976578,0.753550,0.073037,2.976578,1,True,4
5,XGBoost,-125837.361380,-122627.483075,0.236872,0.050016,13.939312,0.236872,0.050016,13.939312,1,True,9
6,LightGBMLarge,-126406.216973,-128412.918788,0.529994,0.096799,6.493626,0.529994,0.096799,6.493626,1,True,11
7,RandomForestMSE,-127408.629182,-133070.626462,2.361191,0.227620,16.746859,2.361191,0.227620,16.746859,1,True,5
8,ExtraTreesMSE,-136689.896601,-140276.706106,0.985898,0.178334,8.092251,0.985898,0.178334,8.092251,1,True,7
9,NeuralNetTorch,-154699.488791,-174614.254381,0.279652,0.030933,26.447577,0.279652,0.030933,26.447577,1,True,10


## 2) Run Autogluon with your hyperparameters

In [9]:
import autogluon.core as ag

In [10]:
eval_metric = 'mae' # setting Mean Absolute Error as evaluation metric

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 20,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

cat_options = {
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  
                   'CAT': cat_options
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 5*60  # train various models for ~5 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=eval_metric, problem_type='regression').fit(
    train_data, time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221128_020420/"
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels/ag-20221128_020420/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3619.09 MB
	Train Data (Original)  Memory Usage: 3.39 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting Dateti

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	-72503.6479	 = Validation score   (-mean_absolute_error)
	0.98s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-71730.751	 = Validation score   (-mean_absolute_error)
	0.79s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-72378.4915	 = Validation score   (-mean_absolute_error)
	1.36s	 = Training   runtime
	0.06s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-157264.605	 = Validation score   (-mean_absolute_error)
	0.96s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	-74963.6513	 = Validation score   (-mean_absolute_error)
	1.35s	 = Training   runtime
	0.06s	 = Validation runtime
Hyperparameter tuning model: CatBoost ... Tuning model for up to 89.93s of the 293.7s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 5197.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost/T1 ...
	-65489.3744	 = Validation score   (-mean_absolute_error)
	89.88s	 = Training   runtime
	0.03s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ... Tuning model for up to 89.93s of the 203.56s of remaining time.
Fitted model: NeuralNetTorch/32dbc_00000 ...
	-493458.1231	 = Validation score   (-mean_absolute_error)
	27.06s	 = Training   runtime
	0.19s	 = Validation runtime
Fitted model: NeuralNetTorch/32dbc_00001 ...
	-76815.7294	 = Validation score   (-mean_absolute_error)
	23.31s	 = Training   runtime
	0.13s	 = Validation runtime
Fitted model: NeuralNetTorch/32dbc_00002 ...
	-125720.4738	 = Validation score   (-mean_absolute_error)
	34.8s	 = Training   runtime
	0.1s	 = Validation runtime
Fitted model: NeuralNetTorch/32dbc_00003 ...
	-85638.6178	 = Validation score   (-mean_absolute_error)
	24.26s	 = Training   runtime
	0.15s	 = Validation runtime
Fi

### Performance evaluation vs default settings

In [11]:
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: mean_absolute_error on test data: -60865.729065054365
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -60865.729065054365
}


## 3) Run Ensembling with your hyperparameters 

In [12]:
# We keep the same Hyperparameters
eval_metric = 'mae'

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 20,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

cat_options = {
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  
                   'CAT': cat_options
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 5*60  # train various models for ~5 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label,
                             eval_metric=eval_metric,
                             problem_type='regression').fit(
    train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = hyperparameters, 
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221128_020702/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221128_020702/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3233.74 MB
	Train Data (Original)  Memory Usage: 3.39 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting DatetimeFeatureGenerator

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L1/T1 ...
	-67127.904	 = Validation score   (-mean_absolute_error)
	1.23s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	-66129.7846	 = Validation score   (-mean_absolute_error)
	1.23s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	-64947.335	 = Validation score   (-mean_absolute_error)
	1.47s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	-153437.2012	 = Validation score   (-mean_absolute_error)
	1.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	-69987.0678	 = Validation score   (-mean_absolute_error)
	1.36s	 = Training   runtime
	0.03s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: CatBoost_BAG_L1/T1 ...
	-61295.9184	 = Validation score   (-mean_absolute_error)
	84.33s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T2 ...
	-62054.8306	 = Validation score   (-mean_absolute_error)
	25.69s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T3 ...
	-60176.9641	 = Validation score   (-mean_absolute_error)
	241.95s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T4 ...
	-60483.8289	 = Validation score   (-mean_absolute_error)
	359.57s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: CatBoost_BAG_L1/T5 ...
	-61580.7449	 = Validation score   (-mean_absolute_error)
	316.98s	 = Training   runtime
	0.04s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ...
Fitted model: NeuralNetTorch_BAG_L1/T1 ...
	-475981.836	 = Validation score   (-mean_absolute_error)
	28.33s	 = Training   runtime
	0.16s	 = Validation runtime
Fitted model

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L2/T1 ...
	-60863.7161	 = Validation score   (-mean_absolute_error)
	0.65s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T2 ...
	-61094.7749	 = Validation score   (-mean_absolute_error)
	0.51s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T3 ...
	-60693.6883	 = Validation score   (-mean_absolute_error)
	0.71s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T4 ...
	-134402.4679	 = Validation score   (-mean_absolute_error)
	0.53s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T5 ...
	-61162.5123	 = Validation score   (-mean_absolute_error)
	0.6s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L2 ...


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: CatBoost_BAG_L2/T1 ...
	-60952.8901	 = Validation score   (-mean_absolute_error)
	4.14s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T2 ...
	-61885.4244	 = Validation score   (-mean_absolute_error)
	2.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T3 ...
	-60968.1895	 = Validation score   (-mean_absolute_error)
	11.46s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T4 ...
	-60939.7042	 = Validation score   (-mean_absolute_error)
	7.58s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T5 ...
	-61017.8538	 = Validation score   (-mean_absolute_error)
	5.02s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L2 ...
Fitted model: NeuralNetTorch_BAG_L2/T1 ...
	-484087.1193	 = Validation score   (-mean_absolute_error)
	16.84s	 = Training   runtime
	0.13s	 = Validation runtime
Fitted model: NeuralNet

### Ensembling Models Leaderboard

In [13]:
predictor.leaderboard(extra_info=True, silent=True).head(5)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,num_features,...,child_model_type,hyperparameters,hyperparameters_fit,ag_args_fit,features,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L2,-61451.825713,1.719316,2020.255072,0.000610,0.576205,2,True,16,7,...,GreedyWeightedEnsembleModel,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[LightGBM_BAG_L1/T2, CatBoost_BAG_L1/T3, NeuralNetTorch_BAG_L1/T2, CatBoost_BAG_L1/T2, CatBoost_BAG_L1/T1, CatBoost_BAG_L1/T4, NeuralNetTorch_BAG_L1/T1]",{'ensemble_size': 100},{'ensemble_size': 100},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[LightGBM_BAG_L1/T2, CatBoost_BAG_L1/T3, CatBoost_BAG_L1/T2, CatBoost_BAG_L1/T1, NeuralNetTorch_BAG_L1/T1, CatBoost_BAG_L1/T4, NeuralNetTorch_BAG_L1/T2]",[]
1,CatBoost_BAG_L1/T3,-62000.277814,0.235343,725.039181,0.235343,725.039181,1,True,8,24,...,CatBoostModel,"{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[condition, date.month, sqft_lot, view, bedrooms, sqft_basement, floors, date.dayofweek, date.year, sqft_lot15, date.day, id, date, bathrooms, yr_renovated, sqft_living15, lat, yr_built, sqft_living, zipcode, long, waterfront, grade, sqft_above]","{'iterations': 10000, 'learning_rate': 0.023861097124304616, 'random_seed': 0, 'allow_writing_files': False, 'eval_metric': 'MAE', 'depth': 8, 'l2_leaf_reg': 3.1795327319875875}",{'iterations': 4928},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None}",[],"[CatBoost_BAG_L2/T1, WeightedEnsemble_L2, LightGBM_BAG_L2/T5, CatBoost_BAG_L2/T3, LightGBM_BAG_L2/T4, NeuralNetTorch_BAG_L2/T2, LightGBM_BAG_L2/T2, NeuralNetTorch_BAG_L2/T3, LightGBM_BAG_L2/T3, LightGBM_BAG_L2/T1, NeuralNetTorch_BAG_L2/T5, CatBoost_BAG_L2/T5, NeuralNetTorch_BAG_L2/T1, WeightedEnsemble_L3, CatBoost_BAG_L2/T2, CatBoost_BAG_L2/T4, NeuralNetTorch_BAG_L2/T4]"
2,WeightedEnsemble_L3,-62069.377452,6.230448,3025.262948,0.000816,0.711148,3,True,32,10,...,GreedyWeightedEnsembleModel,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'drop_unique': False}","[CatBoost_BAG_L2/T1, LightGBM_BAG_L2/T1, CatBoost_BAG_L2/T3, NeuralNetTorch_BAG_L2/T5, NeuralNetTorch_

### Performance evaluation

In [14]:
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: mean_absolute_error on test data: -59758.40548851018
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -59758.40548851018
}


## Extra run with more time

In [21]:
time_limit = 60*60  # train various models for ~1 h
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': 20,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='mae'
                            ).fit(train_data, time_limit=time_limit,
                                  auto_stack=True, presets='best_quality',
                                  hyperparameters = hyperparameters, 
                                  hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
                                 )

No path specified. Models will be saved in: "AutogluonModels/ag-20221128_152246/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20221128_152246/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    15129
Train Data Columns: 20
Label Column: price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7700000.0, 75000.0, 541001.55523, 370160.66869)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipe

  0%|          | 0/20 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L1/T1 ...
	-68032.4084	 = Validation score   (-mean_absolute_error)
	0.44s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	-66465.3303	 = Validation score   (-mean_absolute_error)
	0.44s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	-63996.7949	 = Validation score   (-mean_absolute_error)
	0.57s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	-158025.8675	 = Validation score   (-mean_absolute_error)
	0.44s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	-70389.9508	 = Validation score   (-mean_absolute_error)
	0.51s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T6 ...
	-150220.7035	 = Validation score   (-mean_absolute_error)
	0.53s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T7 ...
	-65536.4852	 = Validation score   (-mean_a

  0%|          | 0/20 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L1/T1 ...
	-60518.5169	 = Validation score   (-mean_absolute_error)
	41.17s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 89.97s of the 3547.75s of remaining time.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
Fitted model: NeuralNetTorch_BAG_L1/T1 ...
	-477350.2494	 = Validation score   (-mean_absolute_error)
	17.23s	 = Training   runtime
	0.07s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T2 ...
	-74092.3509	 = Validation score   (-mean_absolute_error)
	14.43s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T3 ...
	-121646.4086	 = Validation score   (-mean_absolute_error)
	20.05s	 = Training   runtime
	0.07s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T4 ...
	-79862.7232	 = Validation score   (-mean_absolute_error)
	1

  0%|          | 0/20 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L2/T1 ...
	-60511.4017	 = Validation score   (-mean_absolute_error)
	0.74s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T2 ...
	-60507.3987	 = Validation score   (-mean_absolute_error)
	0.67s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T3 ...
	-60214.4798	 = Validation score   (-mean_absolute_error)
	1.01s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T4 ...
	-134794.9051	 = Validation score   (-mean_absolute_error)
	0.69s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T5 ...
	-61114.6521	 = Validation score   (-mean_absolute_error)
	0.81s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T6 ...
	-130929.1272	 = Validation score   (-mean_absolute_error)
	0.89s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L2/T7 ...
	-60769.4851	 = Validation score   (-mean_ab

  0%|          | 0/20 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 1626.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L2/T1 ...
	-60615.4709	 = Validation score   (-mean_absolute_error)
	21.85s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T2 ...
	-62206.9639	 = Validation score   (-mean_absolute_error)
	5.58s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: CatBoost_BAG_L2/T3 ...
	-60629.902	 = Validation score   (-mean_absolute_error)
	44.12s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L2 ... Tuning model for up to 89.47s of the 2297.08s of remaining time.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
Fitted model: NeuralNetTorch_BAG_L2/T1 ...
	-488564.9955	 = Validation score   (-mean_absolute_error)
	15.91s	 = Training   runtime
	0.14s	 = Validation runtime
F

In [22]:
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: mean_absolute_error on test data: -59682.529194459436
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -59682.529194459436
}
